In [2]:
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification


In [8]:
model_path = "/home/nonso/ai-multimodal-learning-project/Finance-Ai-Project/outputs/models/finbert-financial-classifier"

In [4]:
df_eval = pd.read_csv("/home/nonso/ai-multimodal-learning-project/Finance-Ai-Project/data/evaluate/finance_1_data.csv")

In [9]:
model = AutoModelForSequenceClassification.from_pretrained(model_path)

tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-pretrain")

In [6]:
label_encoder = LabelEncoder()
df_eval["label"] = label_encoder.fit_transform(df_eval["label"])

In [7]:
def tokenize(batch):
    return tokenizer(batch["text"], max_length=512, padding=True, truncation=True)

In [11]:
dataset = Dataset.from_pandas(df_eval[["text", "label"]])
dataset = dataset.map(tokenize, batched=True)

dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/48 [00:00<?, ? examples/s]

Evaluate

In [12]:
from transformers import TrainingArguments, Trainer 
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

In [13]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred 
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy_score": accuracy_score(labels, preds),
        "f1_score": f1_score(labels, preds, average="weighted")
    }

In [14]:
training_args = TrainingArguments(
    output_dir="/home/nonso/ai-multimodal-learning-project/Finance-Ai-Project/outputs/models",
    per_device_eval_batch_size=4,
    do_eval=True
)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipykernel_19302/1944674238.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [16]:
results = trainer.evaluate(eval_dataset=dataset)
print(results)

{'eval_loss': 6.493112564086914, 'eval_model_preparation_time': 0.0099, 'eval_accuracy_score': 0.4583333333333333, 'eval_f1_score': 0.4583333333333333, 'eval_runtime': 58.5934, 'eval_samples_per_second': 0.819, 'eval_steps_per_second': 0.205}
